# Kiva collaborative filtering
## Polara framework

***
### Imports

In [30]:
# essentials
import os
import sys
import csv
import pandas as pd
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
from scipy.sparse import csr_matrix, lil_matrix

# polara framework
from polara.recommender.data import RecommenderData
from polara.recommender.models import SVDModel, PopularityModel, RandomModel
from polara.recommender.external.implicit.ialswrapper import ImplicitALS
from polara.datasets.movielens import get_movielens_data
from polara.evaluation import evaluation_engine as ee
from polara.evaluation.plotting import show_hit_rates, show_precision_recall, show_ranking, show_relevance

# utilities
import codecs
import logging
import time
import tqdm

# visualization libraries
import seaborn as sns
import matplotlib.pyplot as plt

# serialization
import pickle

In [2]:
pd.set_option('display.max_columns', 40)

In [3]:
%env MKL_NUM_THREADS=1
logging.basicConfig(level=logging.DEBUG)

env: MKL_NUM_THREADS=1


***
### Pickle loads

In [ ]:
loans_table = pickle.load(open("pickle/loans_table.p", "rb"))
funded_loans_table = pickle.load(open("pickle/funded_loans_table.p", "rb"))
funded_loan_ids_set = pickle.load(open("pickle/funded_loan_ids_set.p", "rb"))

In [ ]:
utility_matrix = pickle.load(open("pickle/utility_matrix.p", "rb"))

In [ ]:
utility_matrix_df = pickle.load(open("pickle/utility_matrix_df.p", "rb"))

In [ ]:
utility_matrix_df.head()

***
### Dataset reading

In [ ]:
loans_table = pd.read_csv('additional-kiva-snapshot/loans.csv')
loans_table = loans_table.sort_values(by='raised_time')

In [ ]:
funded_loans_table = loans_table[loans_table.status == 'funded']

In [ ]:
START_DATE = '2013-10-01'
END_DATE = '2015-05-01'

mask = (funded_loans_table['raised_time'] > START_DATE) & (funded_loans_table['raised_time'] <= END_DATE)
funded_loans_table = funded_loans_table.loc[mask]
del mask

funded_loan_ids_set = set(funded_loans_table['loan_id'])

In [ ]:
# pickle.dump(loans_table, open("/pickle/loans_table.p", "wb"))
# pickle.dump(funded_loans_table, open("pickle/funded_loans_table.p", "wb"))
# pickle.dump(funded_loan_ids_set, open("pickle/funded_loan_ids_set.p", "wb"))

In [ ]:
# free ram
del loans_table
del funded_loans_table

In [ ]:
loans = set()
lenders = set()
loans_lenders_dict = {}

with open('additional-kiva-snapshot/loans_lenders.csv', newline='', encoding="utf8") as csvfile:
    csv_reader = csv.reader(csvfile)
    line_num = 0
    for row in csv_reader:
        if line_num == 0:
            line_num += 1
            continue
        loan_id, lender_ids = row
        loan_id = int(loan_id)
        if loan_id not in funded_loan_ids_set:
            continue
        
        loans.add(loan_id)
        new_lenders = set(lender_ids.split(", "))
        loans_lenders_dict[loan_id] = new_lenders
        lenders.update(new_lenders)
        line_num += 1

loans = list(loans)
lenders = list(lenders)

print('Loans-lenders dict filled')

***
### Utility matrix creation

In [ ]:
lenders_reverse_index = {k: v for v, k in enumerate(lenders)}
utility_matrix = lil_matrix((len(loans), len(lenders)), dtype=np.float64)

In [ ]:
for loan_index, loan in enumerate(loans):
    for lender in loans_lenders_dict[loan]:
        lender_index = lenders_reverse_index[lender]
        utility_matrix[loan_index, lender_index] = 1.0

print('Filled utility matrix')

In [ ]:
utility_matrix = utility_matrix.tocsr()

In [ ]:
# pickle.dump(utility_matrix, open("pickle/utility_matrix.p", "wb"))

#### Conversion to SparseDataFrame

In [ ]:
utility_matrix_df = pd.SparseDataFrame(utility_matrix)

In [ ]:
# pickle.dump(utility_matrix_df, open("pickle/utility_matrix_df.p", "wb"))

In [ ]:
def write_matrix_to_csv(matrix, filename):
    if not isinstance(matrix, scipy.sparse.coo_matrix):
        matrix = matrix.tocoo()
    
    with open(filename, 'w', encoding='utf-8') as f:
        f.write("%s,%s,%s\n" % ('itemid', 'userid', 'feedback'))
        for i,j,v in zip(matrix.row, matrix.col, matrix.data):
            f.write("%s,%s,%s\n" % (i, j, v))

In [4]:
kiva_dataframe = pd.read_csv('kiva_dataframe.csv', engine='c')

In [5]:
kiva_dataframe.shape

(5339455, 3)

In [8]:
kiva_dataframe.head()

,itemid,userid,feedback
0,0,7703,1.0
1,0,8354,1.0
2,0,9000,1.0
3,0,10247,1.0
4,0,21284,1.0


***
### Polara example

In [ ]:
# define models
ml_data = get_movielens_data(get_genres=False)
data_model = RecommenderData(ml_data, 'userid', 'movieid', 'rating')

# data_model = RecommenderData(kiva_dataframe, 'userid', 'itemid', 'feedback')

svd = SVDModel(data_model)
popular = PopularityModel(data_model)
random = RandomModel(data_model)
models = [svd, popular, random]

metrics = ['ranking', 'relevance'] # metrics for evaluation: NDGC, Precision, Recall, etc.
folds = [1, 2, 3, 4, 5] # use all 5 folds for cross-validation (default)
topk_values = [1, 5, 10, 20, 50] # values of k to experiment with

# run 5-fold CV experiment
result = ee.run_cv_experiment(models, folds, metrics,
                              fold_experiment=ee.topk_test,
                              topk_list=topk_values)

In [ ]:
# calculate average values across all folds for e.g. relevance metrics
scores = result.mean(axis=0, level=['top-n', 'model']) # use .std instead of .mean for standard deviation
scores.xs('nDCG', level='metric', axis=1).unstack('model')

***
### Cross-validation

In [9]:
data_model = RecommenderData(kiva_dataframe, 'userid', 'itemid', 'feedback', seed=0)
data_model.warm_start = False

start = time.time()
data_model.prepare()
logging.debug("Prepared data in %0.2fs", time.time() - start)

Preparing data...
207 unique itemid's within 228 holdout interactions were filtered. Reason: not in the training data.
85452 unique userid's within 136096 holdout interactions were filtered. Reason: not in the training data.
114 of 44225 userid's were filtered out from holdout. Reason: incompatible number of items.
Done.
There are 5070574 events in the training and 132333 events in the holdout.


DEBUG:root:Prepared data in 144.29s


In [38]:
als_default = ImplicitALS(data_model) # create model
als_default.rank = 10
als_default.alpha = 1
als_default.epsilon = 1
als_default.weight_func = None
als_default.regularization = 0.1
als_default.num_threads = 0
als_default.num_epochs = 30

# als_default.build() # fit model

# start = time.time()
# als_default.evaluate() # by default it calculates the total number of hits
# logging.debug("Evaluated model ALS in %0.2fs", time.time() - start)

In [39]:
als_alpha_forty = ImplicitALS(data_model) # create model
als_alpha_forty.rank = 10
als_alpha_forty.alpha = 40
als_alpha_forty.epsilon = 1
als_alpha_forty.weight_func = None
als_alpha_forty.regularization = 0.1
als_alpha_forty.num_threads = 0
als_alpha_forty.num_epochs = 30

# als_alpha_forty.build() # fit model

# start = time.time()
# als_alpha_forty.evaluate() # by default it calculates the total number of hits
# logging.debug("Evaluated model ALS in %0.2fs", time.time() - start)

In [40]:
popular = PopularityModel(data_model)
random = RandomModel(data_model)

In [41]:
models = [als_default, popular, random, als_alpha_forty]

metrics = ['ranking', 'relevance'] # metrics for evaluation: NDGC, Precision, Recall, etc.
folds = [1, 2, 3] # use all 5 folds for cross-validation (default)
topk_values = [5, 10, 20] # values of k to experiment with

start = time.time()

# run 5-fold CV experiment
result = ee.run_cv_experiment(models, folds, metrics,
                              fold_experiment=ee.topk_test,
                              topk_list=topk_values)


logging.debug("Cross-validation experiment finished in %0.2fs", time.time() - start)

DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.187s
DEBUG:implicit:Calculated transpose in 0.125s
DEBUG:implicit:Initialized factors in 0.4373898506164551
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:10<00:00,  2.87it/s]


iALS training time: 13.195s


DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.172s
DEBUG:implicit:Calculated transpose in 0.141s
DEBUG:implicit:Initialized factors in 0.40612268447875977
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:10<00:00,  2.91it/s]


iALS training time: 11.873s
Preparing data...
182 unique itemid's within 197 holdout interactions were filtered. Reason: not in the training data.
85704 unique userid's within 136225 holdout interactions were filtered. Reason: not in the training data.
89 of 43984 userid's were filtered out from holdout. Reason: incompatible number of items.
Done.
There are 5071181 events in the training and 131685 events in the holdout.


DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.172s
DEBUG:implicit:Calculated transpose in 0.141s
DEBUG:implicit:Initialized factors in 0.4373960494995117
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:11<00:00,  2.91it/s]


...

DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.401s
DEBUG:implicit:Calculated transpose in 0.200s
DEBUG:implicit:Initialized factors in 0.48560595512390137
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:10<00:00,  2.95it/s]


iALS training time: 13.563s


DEBUG:implicit:Converting input to CSR format
DEBUG:implicit:Converted input to CSR in 0.232s
DEBUG:implicit:Calculated transpose in 0.200s
DEBUG:implicit:Initialized factors in 0.48564767837524414
DEBUG:implicit:Running 30 ALS iterations
100%|████████████████████████████████████████████████████████████████████████████████| 30.0/30 [00:10<00:00,  2.89it/s]


iALS training time: 12.101s


DEBUG:root:Cross-validation experiment finished in 2771.97s


In [ ]:
# holdout_sizes = [1, 2, 5]

# result = ee.run_cv_experiment(models, folds, metrics,
#                               fold_experiment=ee.holdout_test,
#                               holdout_sizes=holdout_sizes)

In [42]:
# calculate average values across all folds for e.g. relevance metrics
scores = result.mean(axis=0, level=['top-n', 'model']) # use .std instead of .mean for standard deviation
# scores.xs('recall', level='metric', axis=1).unstack('model')
scores

type        relevance                       ranking
metric      precision    recall miss_rate      nDCG
top-n model                                        
5     iALS   0.001109  0.001849  0.998151  0.001528
      MP     0.001156  0.001927  0.998073  0.001675
      RND    0.000014  0.000023  0.999977  0.000016
      iALS   0.001353  0.002255  0.997745  0.001904
10    iALS   0.000967  0.003223  0.996777  0.002154
      MP     0.000841  0.002804  0.997196  0.002052
      RND    0.000011  0.000035  0.999965  0.000022
      iALS   0.001272  0.004239  0.995761  0.002801
20    iALS   0.000727  0.004847  0.995153  0.002727
      MP     0.000909  0.006063  0.993937  0.003236
      RND    0.000014  0.000091  0.999909  0.000041
      iALS   0.001072  0.007146  0.992854  0.003832

In [ ]:
show_relevance(result)

In [ ]:
ml_data = get_movielens_data(get_genres=False)
ml_data

***
### Variable sizes

In [32]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' By Fred Cirera, after https://stackoverflow.com/a/1094933/1870254'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name,value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name,sizeof_fmt(size)))

                kiva_dataframe: 122.2MiB
                             _:   4.3KiB
                        result:   4.3KiB
                           _20:   4.3KiB
               RecommenderData:   2.0KiB
                   ImplicitALS:   2.0KiB
               PopularityModel:   1.4KiB
                   RandomModel:   1.4KiB
                    lil_matrix:   1.0KiB
                      SVDModel:   1.0KiB
